In [3]:
!pip -q install ipywidgets scipy
from google.colab import output
output.enable_custom_widget_manager()



In [4]:
%%writefile utils/constants.py
import numpy as np

q      = 1.602176634e-19        # C
k_B    = 1.380649e-23           # J/K
T0     = 300.0                  # K  (default)
kT_eV  = lambda T=T0: (k_B*T)/q # eV
eps0   = 8.8541878128e-14       # F/cm
eps_si = 11.7 * eps0            # Silicon permittivity
Ni_300 = 1.0e10                 # cm⁻³  (approx 300 K)
hbar   = 6.582119569e-16        # eV·s
m0     = 9.10938356e-31         # kg


Writing utils/constants.py


In [5]:
%%writefile mesh.py
import numpy as np

def build_1d_mesh(L=1e-4, N=601):
    x  = np.linspace(0.0, L, N)
    dx = x[1] - x[0]
    return x, dx



Writing mesh.py


In [6]:
%%writefile models/mobility.py
import numpy as np
from utils.constants import q

def mu_lattice(N, mu_min, mu_max, Nref, alpha):
    return mu_min + (mu_max - mu_min)/(1.0 + (N/Nref)**alpha)

def mu_electron_lattice(N):    # cm² V⁻¹ s⁻¹
    return mu_lattice(N, 55, 1417, 8.5e16, 0.68)

def mu_hole_lattice(N):        # cm² V⁻¹ s⁻¹
    return mu_lattice(N, 44, 470, 6.3e16, 0.76)


def mu_field(mu0, E, v_sat=1e7):
    return mu0 / (1.0 + mu0*np.abs(E)/v_sat)

def mobility(n, p, E, Ndop):
    Ntot = np.abs(Ndop) + n + p
    mu_n0 = mu_electron_lattice(Ntot)
    mu_p0 = mu_hole_lattice(Ntot)
    mu_n  = mu_field(mu_n0, E)
    mu_p  = mu_field(mu_p0, E)
    return mu_n, mu_p



Writing models/mobility.py


In [7]:
%%writefile models/recombination.py
import numpy as np
from utils.constants import Ni_300, q

def SRH(n, p, tau_n=1e-6, tau_p=1e-6, Ni=Ni_300):
    return (n*p - Ni**2) / (tau_p*(n + Ni) + tau_n*(p + Ni))

def Auger(n, p, Cn=2.8e-31, Cp=9.9e-32):
    return (Cn*n + Cp*p)*(n*p - Ni_300**2)



Writing models/recombination.py


In [8]:
%%writefile models/impact_ion.py
import numpy as np

def selberherr_alpha(E, alpha0=7e5, Ecrit=1e5, beta=1.0):

    E = np.maximum(np.abs(E), 1e3)
    return alpha0 * np.exp(-(Ecrit/E)**beta)


Writing models/impact_ion.py


In [9]:
%%writefile poisson.py
import numpy as np, scipy.sparse as sp, scipy.sparse.linalg as spla
from utils.constants import q, eps_si, kT_eV, Ni_300

def _rho(phi, Ndop, T):
    n = Ni_300*np.exp( +q*phi/kT_eV(T) )
    p = Ni_300*np.exp( -q*phi/kT_eV(T) )
    return q*(p - n + Ndop)

def solve_poisson(phi, Ndop, dx, T=300, tol=1e-6, maxit=50):
    N = phi.size
    for it in range(maxit):
        main = np.zeros(N)
        off  = np.full(N-1, eps_si/dx**2)
        R    = np.zeros(N)

        for i in range(1, N-1):
            main[i] = -2*eps_si/dx**2
            R[i]    = eps_si*(phi[i-1]-2*phi[i]+phi[i+1])/dx**2 - _rho(phi[i], Ndop[i], T)
        main[0] = main[-1] = 1.0  # Dirichlet φ=0 at contacts
        R[0] = R[-1] = 0.0

        J = sp.diags([off, main, off], [-1,0,1], format='csc')
        dphi = spla.spsolve(J, -R)
        phi += dphi
        if np.max(np.abs(dphi)) < tol:
            break
    return phi



Writing poisson.py


In [10]:
%%writefile drift_diffusion.py
import numpy as np
from utils.constants import q, kT_eV, Ni_300
from models.mobility import mobility
from models.recombination import SRH, Auger
from models.impact_ion import selberherr_alpha

def equilibrium_carriers(phi, T):
    n = Ni_300 * np.exp(+q*phi/kT_eV(T))
    p = Ni_300 * np.exp(-q*phi/kT_eV(T))
    return n, p

def postprocess(phi, Ndop, dx, T=300):
    n, p = equilibrium_carriers(phi, T)

    # Electric field
    E = -np.gradient(phi, dx)

    # Mobility
    mu_n, mu_p = mobility(n, p, E, Ndop)

    # Recombination & impact
    R_srh   = SRH(n, p)
    R_auger = Auger(n, p)
    alpha   = selberherr_alpha(E)
    G_ii    = alpha*(q*mu_n*n*E + q*mu_p*p*E)/q  # cm⁻3 s⁻1

    Gnet = -R_srh - R_auger + G_ii

    return n, p, E, mu_n, mu_p, Gnet



Writing drift_diffusion.py


In [11]:
%%writefile mosfet_gui.py
import numpy as np, matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatLogSlider, VBox, HBox

from mesh import build_1d_mesh
from poisson import solve_poisson
from drift_diffusion import postprocess
from utils.constants import eps_si


def build_doping(x, NA, ND, Ls, Ld):
    L = x[-1]
    Ndop = np.zeros_like(x)
    Ndop[x < Ls] = ND
    Ndop[x > L-Ld] = ND
    Ndop[(x >= Ls) & (x <= L-Ld)] = -NA
    return Ndop


def simulate(Vg, NA, ND, L, T):
    x, dx = build_1d_mesh(L=L)
    Ndop  = build_doping(x, NA, ND, 5e-6, 5e-6)


    phi_init = np.zeros_like(x); phi_init += Vg
    phi = solve_poisson(phi_init, Ndop, dx, T=T)

    n, p, E, mu_n, mu_p, Gnet = postprocess(phi, Ndop, dx, T=T)
    return x, phi, n, p, Ndop, E, mu_n, mu_p, Gnet


def plot_all(x, phi, n, p, Nd, E, mu_n, mu_p, Gnet, Vg):
    fig, ax = plt.subplots(3,1, figsize=(8,10), sharex=True)

    # --- φ & E ---
    ax[0].plot(x*1e4, phi, label='φ (V)')
    ax0b = ax[0].twinx()
    ax0b.plot(x*1e4, E/1e4, 'r--', label='E (kV/cm)')
    ax0b.set_ylabel('E  (kV/cm)', color='r')
    ax[0].set_ylabel('Potential φ (V)')
    ax[0].set_title(f'1‑D MOSFET | Vg = {Vg:.2f} V')
    ax[0].grid(True)

    # --- carrier densities & doping ---
    ax[1].semilogy(x*1e4, n, label='n')
    ax[1].semilogy(x*1e4, p, label='p')
    ax[1].semilogy(x*1e4, np.abs(Nd),'k--', label='|Ndop|')
    ax[1].set_ylabel('cm$^{-3}$')
    ax[1].grid(True, which='both', ls='--')
    ax[1].legend()

    # --- mobility & net G ---
    ax[2].plot(x*1e4, mu_n, label='μn')
    ax[2].plot(x*1e4, mu_p, label='μp')
    ax2b = ax[2].twinx()
    ax2b.semilogy(x*1e4, np.abs(Gnet)+1e-30, 'm--', label='|G_net|')
    ax2b.set_ylabel('|G_net|  (cm$^{-3}$ s$^{-1}$)', color='m')
    ax[2].set_xlabel('x (µm)'); ax[2].set_ylabel('μ (cm² V⁻¹ s⁻¹)')
    ax[2].grid(True)
    ax[2].legend(loc='upper left')

    plt.tight_layout(); plt.show()


def gui():
    Vg  = FloatSlider(0, min=-1, max=2, step=0.05, description='Vg (V)')
    NA  = FloatLogSlider(1e16, base=10, min=14, max=18, description='NA body')
    ND  = FloatLogSlider(1e19, base=10, min=17, max=20, description='ND S/D')
    L   = FloatLogSlider(1e-4, base=10, min=-5, max=-3, description='L (cm)')
    Temp= FloatSlider(300, min=250, max=400, step=10, description='T (K)')

    def _update(Vg, NA, ND, L, Temp):
        x, φ, n, p, Nd, E, mu_n, mu_p, G = simulate(Vg, NA, ND, L, Temp)
        plot_all(x, φ, n, p, Nd, E, mu_n, mu_p, G, Vg)

    interact(_update, Vg=Vg, NA=NA, ND=ND, L=L, Temp=Temp)


try:
    get_ipython(); gui()
except Exception:
    pass


Writing mosfet_gui.py


In [12]:
import mosfet_gui



interactive(children=(FloatSlider(value=0.0, description='Vg (V)', max=2.0, min=-1.0, step=0.05), FloatLogSlid…